# Importando libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

# Lendo o dataset

In [2]:
df = pd.read_csv('../data/prouni_2005_2019_processed.csv')
df

,ANO DE CONCESSÃO,IES,TIPO,MODALIDADE DE ENSINO,CURSO,TURNO,SEXO,RAÇA,DEFICIENTE FÍSICO,REGIÃO,UF,MUNICÍPIO,IDADE
0,2005,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Enfermagem,Integral,F,Branca,nao,SUL,RS,santo angelo,34.0
1,2005,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Servico Social,Noturno,F,Parda,nao,SUL,RS,frederico westphalen,35.0
2,2005,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Servico Social,Noturno,F,Parda,nao,SUL,RS,frederico westphalen,37.0
3,2005,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Ciencia Da Computacao,Noturno,M,Branca,nao,SUL,RS,frederico westphalen,33.0
4,2005,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Ciencia Da Computacao,Noturno,M,Amarela,nao,SUL,RS,frederico westphalen,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685913,2019,FACULDADE REBOUAAS DE CAMPINA GRANDE,BOLSA INTEGRAL,PRESENCIAL,FarmAcia,Noturno,F,Parda,nao,NORDESTE,PE,santa terezinha,20.0
2685914,2019,FACULDADE REBOUAAS DE CAMPINA GRANDE,BOLSA INTEGRAL,PRESENCIAL,GestAo De Recursos Humanos,Matutino,F,Parda,nao,NORDESTE,PB,aroeiras,20.0
2685915,2019,FACULDADE REBOUAAS DE CAMPINA GRANDE,BOLSA INTEGRAL,PRESENCIAL,GestAo De Recursos Humanos,Matutino,M,Branca,nao,NORDESTE,PB,campina grande,28.0
2685916,2019,FACULDADE REBOUAAS DE CAMPINA GRANDE,BOLSA INTEGRAL,PRESENCIAL,GestAo De Recursos Humanos,Noturno,F,Parda,nao,NORDESTE,PB,lagoa seca,24.0


# Gráficos

In [3]:
df_num_bolsas = df.copy()

In [4]:
df_idades_90 = (
    df_num_bolsas[df_num_bolsas['IDADE'] > 90]['IDADE']
    .value_counts()
    .reset_index()
)

df_idades_90.columns = ['IDADE', 'Quantidade']

df_idades_90 = df_idades_90.sort_values('IDADE')

df_idades_90.sum()

IDADE         770.0
Quantidade     17.0
dtype: float64

In [5]:
lista = sorted(df_num_bolsas['IDADE'].unique())
df_lista = pd.DataFrame(lista, columns=["IDADE"])
df_lista

,IDADE
0,2.0
1,5.0
2,8.0
3,9.0
4,10.0
...,...
88,96.0
89,97.0
90,98.0
91,99.0


## Número de bolsas por ano

In [6]:
anos = sorted(df_num_bolsas["ANO DE CONCESSÃO"].unique())
dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

def plot_tipo_bolsas(ano):
    dados = df_num_bolsas[df_num_bolsas["ANO DE CONCESSÃO"] == ano]
    
    # contagem de tipos de bolsa nesse ano
    contagem = (
        dados["TIPO"]
        .value_counts()
        .reset_index()
    )
    contagem.columns = ["TIPO", "QUANTIDADE"]
    
    fig = px.bar(
        contagem,
        x="TIPO",
        y="QUANTIDADE",
        text="QUANTIDADE",
        title=f"Quantidade de cada tipo de bolsa concedida em {ano}",
    )
    
    # deixa o texto em cima da barra e melhora layout
    fig.update_traces(textposition="outside")
    fig.update_layout(
        height= 650,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE",
        uniformtext_minsize=2,
        uniformtext_mode="hide"
    )
    
    fig.show()

widgets.interact(plot_tipo_bolsas, ano=dropdown_ano);


interactive(children=(Dropdown(description='ANO', options=(np.int64(2005), np.int64(2006), np.int64(2007), np.…

## Número de bolsas por sexo

In [7]:
import ipywidgets as widgets
import plotly.express as px

anos = sorted(df_num_bolsas["ANO DE CONCESSÃO"].unique())
dropdown_ano = widgets.Dropdown(
    options=anos,
    value=anos[0],
    description="ANO"
)

def plot_barras_total_e_sexo(ano):
    # Filtra o ano desejado
    dados = df_num_bolsas[df_num_bolsas["ANO DE CONCESSÃO"] == ano]
    
    # Agrupa por tipo e sexo
    agrupado = (
        dados.groupby(["TIPO", "SEXO"])
             .size()
             .reset_index(name="QUANTIDADE")
    )
    
    # Tabela com colunas separadas para F e M
    tabela = agrupado.pivot_table(
        index="TIPO",
        columns="SEXO",
        values="QUANTIDADE",
        fill_value=0
    ).reset_index()
    
    # Garante que existam as colunas F e M
    if "F" not in tabela.columns:
        tabela["F"] = 0
    if "M" not in tabela.columns:
        tabela["M"] = 0
    
    # Calcula total
    tabela["TOTAL"] = tabela["F"] + tabela["M"]
    
    # Arruma nomes das colunas
    tabela.columns.name = None
    tabela = tabela.rename(columns={
        "F": "FEMININO",
        "M": "MASCULINO"
    })
    
    # Gráfico de barras com TOTAL, FEMININO e MASCULINO
    fig = px.bar(
        tabela,
        x="TIPO",
        y=["TOTAL", "FEMININO", "MASCULINO"],
        barmode="group",
        title=f"Quantidade de bolsas por tipo e sexo em {ano}",
        text_auto=True,
        color_discrete_map={
            "TOTAL": "#444444",
            "FEMININO": "#e83e8c",
            "MASCULINO": "#1f77b4"
        }
    )
    
    fig.update_layout(
        height=650,
        xaxis_title="TIPO DE BOLSA",
        yaxis_title="QUANTIDADE DE BOLSAS",
        uniformtext_minsize=6,
        uniformtext_mode="hide"
    )
    
    fig.show()

widgets.interact(plot_barras_total_e_sexo, ano=dropdown_ano);


interactive(children=(Dropdown(description='ANO', options=(np.int64(2005), np.int64(2006), np.int64(2007), np.…

## Crescimento de cada tipo de bolsa ao longo dos anos

In [8]:
df_crescimento = (
   df_num_bolsas
   .groupby(["ANO DE CONCESSÃO", "TIPO"])
   .size()
   .reset_index(name="Quantidade")
)

In [9]:
# Paleta opcional por tipo
cores_por_tipo = {
    "BOLSA INTEGRAL": "#1f77b4",
    "BOLSA PARCIAL 50%": "#ff7f0e",
    "BOLSA COMPLEMENTAR 25%": "#2ca02c"
}

# Dropdowns
tipos = sorted(df_num_bolsas["TIPO"].unique())
dropdown_tipo = widgets.Dropdown(options=tipos, value=tipos[0], description="TIPO")

# PAÍS = agregado, Comparar regiões = várias linhas
regioes = ["PAÍS", "COMPARAR REGIÕES"] + sorted(df_num_bolsas["REGIÃO"].unique())
dropdown_regiao = widgets.Dropdown(options=regioes, value="PAÍS", description="REGIÃO")

modos = ["QUANTIDADE", "CRESCIMENTO PERCENTUAL"]
dropdown_modo = widgets.Dropdown(options=modos, value="QUANTIDADE", description="ESCALA")


def plot_crescimento(tipo, regiao, modo):
    base = df_num_bolsas[df_num_bolsas["TIPO"] == tipo].copy()
    if base.empty:
        print("Sem dados para esse tipo.")
        return

    # Cenário 1: PAÍS (agregado)
    if regiao == "PAÍS":
        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="Quantidade")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO"] = tipo

        primeiro_valor = dados["Quantidade"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["Quantidade"] / primeiro_valor - 1) * 100

        if modo == "Quantidade":
            metrica = "Quantidade"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} no país",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO": True,
                "Quantidade": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "Quantidade" else False,
            }
        )

    # Cenário 2: região específica
    elif regiao in base["REGIÃO"].unique():
        base = base[base["REGIÃO"] == regiao]
        if base.empty:
            print("Sem dados para essa região.")
            return

        dados = (
            base.groupby("ANO DE CONCESSÃO")
                .size()
                .reset_index(name="Quantidade")
                .sort_values("ANO DE CONCESSÃO")
        )
        dados["TIPO"] = tipo
        dados["REGIÃO"] = regiao

        primeiro_valor = dados["Quantidade"].iloc[0]
        if primeiro_valor == 0:
            dados["Crescimento_percentual"] = pd.NA
        else:
            dados["Crescimento_percentual"] = (dados["Quantidade"] / primeiro_valor - 1) * 100

        if modo == "Quantidade":
            metrica = "Quantidade"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano (%)"

        cor_tipo = cores_por_tipo.get(tipo, "#1f77b4")

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            markers=True,
            title=f"Evolução de {tipo} na região {regiao}",
            color_discrete_sequence=[cor_tipo],
            hover_data={
                "ANO DE CONCESSÃO": True,
                "TIPO": True,
                "REGIÃO": True,
                "Quantidade": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "Quantidade" else False,
            }
        )

    # Cenário 3: Comparar regiões
    elif regiao == "COMPARAR REGIÕES":
        dados = (
            base.groupby(["ANO DE CONCESSÃO", "REGIÃO"])
                .size()
                .reset_index(name="Quantidade")
                .sort_values(["REGIÃO", "ANO DE CONCESSÃO"])
        )

        # Crescimento percentual por região
        def calc_crescimento(s):
            if s.iloc[0] == 0:
                return pd.Series([pd.NA] * len(s), index=s.index)
            return (s / s.iloc[0] - 1) * 100

        dados["Crescimento_percentual"] = (
            dados.groupby("REGIÃO")["Quantidade"].transform(calc_crescimento)
        )

        if modo == "Quantidade":
            metrica = "Quantidade"
            y_label = "Quantidade de bolsas"
        else:
            metrica = "Crescimento_percentual"
            y_label = "Crescimento em relação ao primeiro ano de cada região (%)"

        fig = px.line(
            dados,
            x="ANO DE CONCESSÃO",
            y=metrica,
            color="REGIÃO",
            markers=True,
            title=f"Evolução de {tipo} comparando regiões",
            hover_data={
                "ANO DE CONCESSÃO": True,
                "REGIÃO": True,
                "Quantidade": ':.0f',
                "Crescimento_percentual": ':.1f' if modo != "Quantidade" else False,
            }
        )

    else:
        print("Opção de região inválida.")
        return

    fig.update_layout(
        xaxis_title="ANO DE CONCESSÃO",
        yaxis_title=y_label,
        xaxis=dict(dtick=1),
        height=650
    )
    fig.show()


widgets.interact(
    plot_crescimento,
    tipo=dropdown_tipo,
    regiao=dropdown_regiao,
    modo=dropdown_modo
);


interactive(children=(Dropdown(description='TIPO', options=('BOLSA COMPLEMENTAR 25%', 'BOLSA INTEGRAL', 'BOLSA…

## Mapa

In [10]:
import json
# Carrega o geojson local (sem internet)
with open("brazil-states.geojson", encoding="utf-8") as f:
   geojson = json.load(f)

# Dropdown de anos
anos = sorted(df_num_bolsas["ANO DE CONCESSÃO"].unique())
dropdown_ano_mapa = widgets.Dropdown(
   options=anos,
   value=anos[0],
   description="ANO"
)

# Dropdown de tipos (com opção "Todos")
tipos = ["Todos os tipos"] + sorted(df_num_bolsas["TIPO"].unique())
dropdown_tipo_mapa = widgets.Dropdown(
   options=tipos,
   value="Todos os tipos",
   description="TIPO"
)

def mapa_bolsas_por_uf(ano, tipo):
   # Filtra por ano
   dados = df_num_bolsas[df_num_bolsas["ANO DE CONCESSÃO"] == ano].copy()
   
   # Filtra por tipo, se não for "Todos"
   if tipo != "Todos os tipos":
      dados = dados[dados["TIPO"] == tipo]
   
   if dados.empty:
      print("Sem dados para essa combinação de ano e tipo.")
      return
   
   # Agrupa por UF e REGIAO
   mapa_df = (
      dados.groupby(["UF", "REGIÃO"])
            .size()
            .reset_index(name="TOTAL")
            .sort_values("TOTAL", ascending=False)
   )

   # Título dinâmico
   if tipo == "Todos os tipos":
      titulo = f"Total de bolsas concedidas por estado em {ano}"
   else:
      titulo = f"Total de bolsas de {tipo} por estado em {ano}"

   # Mapa coroplético por estado
   fig = px.choropleth(
      mapa_df,
      geojson=geojson,
      locations="UF",                  # sua coluna de UF, tipo "SC", "SP"
      featureidkey="properties.sigla", # campo de UF dentro do geojson
      color="TOTAL",
      color_continuous_scale="Blues",
      hover_data={"UF": True, "REGIÃO": True, "TOTAL": True},
      title=titulo
   )

   fig.update_geos(
      fitbounds="locations",
      visible=False
   )

   fig.update_layout(
      height=650,
      coloraxis_colorbar=dict(title="TOTAL DE BOLSAS"),
      margin={"r":0, "t":50, "l":0, "b":0}
   )

   fig.show()

widgets.interact(
   mapa_bolsas_por_uf,
   ano=dropdown_ano_mapa,
   tipo=dropdown_tipo_mapa
);


interactive(children=(Dropdown(description='ANO', options=(np.int64(2005), np.int64(2006), np.int64(2007), np.…